In [281]:
# Change the width of the notebook cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100!important; }</style>"))

# A below code causes ipython's completer to bypass jedi and speeds up the
# loading time of the autocomplete menu to fix the unprompted autocomplete.
# %config Completer.use_jedi = False

# Adding Latitude and Longitude Coordinates

## **Objective**

Find the geographic location of each headline in latitude and longitude coordinates from the city/country names.
## **Workflow**

1. Load in the pandas DataFrame with headline, countries, and cities.
1. If a headline contains multiple cities/countries, decide which single one to keep.
4. For each city/country, match the name to the latitude and longitude in geonamescache.
4. You can use the function gc.get_cities_by_names_ _(“city_name”).
4. Some cities will return multiple matches with the previous function in different countries. You’ll have to decide which city to keep based on a heuristic (rule of thumb).
4. If you have trouble, work with a single problematic city until you figure it out, then write a function to apply on all headlines.
4. Add longitude and latitude coordinates to your DataFrame for each headline.
4. It will be helpful to get the countrycode of each headline at this point.
4. If you were not able to find many countries, think about dropping the column. You also need to decide what to do with headlines that have no coordinates.
4. You should end up with over 600 headlines that have geographic coordinates.

## **Importance to project**

In this section, we are augmenting the original data with external information. Our hypothesis is that there may be groups of headlines in close proximity referring to similar diseases. We will use the locations from this part to test our hypothesis by clustering headlines in the next part.

If we just searched the headlines for diseases, we might find repeated but geographically unrelated (not close to each other) mentions. These could lead to false positives that are only isolated cases.

## **Notes**

Make sure you are inspecting your data statistically and visually at each step. Compute summary statistics and make basic distribution plots (histograms) to search for anomalies.
When you find issues (we all make mistakes), go back and correct them. You may find yourself repeatedly carrying out data cleaning operations; this is normal for a data science project!

Keep in mind that the more accurate the geographic locations of the headlines, the better will be the clustering and the overall conclusions. Checking your data one more time is worth it when lives are at stake.

## **Submit Your Work**

The deliverable is a Jupyter Notebook documenting your work as you add three additional columns to the DataFrame: longitude, latitude, and countrycode. We will use these coordinates to cluster the headlines in the next section. An example of the structure and content of the expected DataFrame is below:

<img src="Picture3_V3.png" />

Upload a link to your Jupyter Notebook (preferably hosted on GitHub) in the blank below and hit submit. After submitting, you can view an example solution in the next section.

# Import all relevent libraries

In [316]:
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Regular expression
import re

import geonamescache

# Data analysis and wrangling
import numpy as np
import pandas as pd

# Set plotting backend options for pandas
pd.options.plotting.backend = "plotly"

# Set display options for pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 200)
# To set the float precision(the number of places after the decimal)
pd.set_option('precision', 4)

# Normalized unicode data (to remove accents)
from unidecode import unidecode
 
## Visualization
# matplotlib
import matplotlib.pyplot as plt
get_ipython().magic('matplotlib inline')
import seaborn as sns
# plotly express
import plotly.express as px
# Set default template
import plotly.io as pio
pio.templates.default = "simple_white"
# Ignore warning
import warnings
warnings.filterwarnings('ignore')

# **Functions**

In [283]:
def cities():
    '''
    '''
    gc = geonamescache.GeonamesCache()

    # Retrive city names data and create a dataframe.
    cities = pd.DataFrame(gc.get_cities()).T.reset_index(drop=True)
    cities = cities.sort_values(by='name').reset_index(drop=True)
    # gc.get_cities_by_name('name')
    # transform all accented strings to English alphabets'
    for index, city in enumerate(cities.loc[:, 'name']):
        cities.loc[index, 'name'] = unidecode(city)
    # Rename name collumn
    cities.rename({'name': 'city'}, axis=1, inplace=True)
    
    # View dataframe
    print(f'cities shape = {cities.shape}')
    cities.population=cities.population.map('{:,.0f}'.format)
   
    display(cities.sample(5))
    print('\n')

    return cities


def countries():
    '''
    '''
    gc = geonamescache.GeonamesCache()
    # Retrive country names data and create a dataframe.
    countries = pd.DataFrame(
        gc.get_countries_by_names()).T.reset_index(drop=True)
    countries = countries.sort_values(by='name',
                                      ascending=False).reset_index(drop=True)
    # countries.info()
    # Rename name collumn
    countries.rename({'name': 'country'}, axis=1, inplace=True)
    countries.population=countries.population.map('{:,.0f}'.format)
    countries.areakm2=countries.areakm2.map('{:,.0f}'.format)
    # View dataframe
    print(f'countries shape = {countries.shape}')
    display(countries.head())
    print('\n')

    return countries


def states():
    '''
    '''
    gc = geonamescache.GeonamesCache()
    # Retrive country names data and create a dataframe.
    states = pd.DataFrame(gc.get_us_states_by_names()).T.reset_index(drop=True)
    states = states.sort_values(by='name',
                                ascending=False).reset_index(drop=True)
    # Rename name collumn
    states.rename({'name': 'state'}, axis=1, inplace=True)
    # View dataframe
    print(f'states shape = {states.shape}')
    display(states.head())
    print('\n')

    return states


def counties():
    '''
    '''
    gc = geonamescache.GeonamesCache()
    # Retrive county names data and create a dataframe.
    counties = pd.DataFrame(gc.get_us_counties()).T.reset_index(drop=True).T
    # Rename columns
    counties.columns = ['code', 'name', 'state']

    counties = (counties.sort_values(by='state',
                                     ascending=False).reset_index(drop=True))
    # Remove general suffixs from county names have only certain names
    s = [
        'County', 'Municipio', 'Island', 'Census Area', 'City and Borough',
        'Borough', 'Parish'
    ]
    regexs = '|'.join(s)

    column = []
    co = []
    counties['county'] = pd.Series()

    for county in counties.name:
        if type(counties['county']) != str:
            compiled_uscounty = re.compile(regexs)
            name_only = compiled_uscounty.sub('', county)
            co.append(name_only)
            column.append(co)

    counties['county'] = pd.Series(co)
    counties = counties.sort_values(by='state')

    # View dataframe
    print(f'counties shape = {counties.shape}')
    display(counties.head())
    print('\n')

    return counties


def us_city():
    '''
    '''
    state_dict = dict(zip(states.code, states.name))

    us_city = (cities[cities.countrycode == 'US'][[
        'name', 'latitude', 'longitude', 'countrycode', 'population',
        'admin1code'
    ]].rename({
        'admin1code': 'us_state',
        'name': 'city'
    }, axis=1))
    us_city['statecode'] = us_city.us_state.values

    us_city.us_state = us_city.us_state.replace(state_dict)

    us_city = us_city[[
        'city', 'us_state', 'population', 'countrycode', 'statecode',
        'latitude', 'longitude'
    ]]

    print(f'us_city shape = {us_city.shape}')
    display(us_city.head())
    print('\n')

    return us_city

def color_null_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for NaN
    , black otherwise.
    """
    color = 'red' if str(val) == 'nan' else 'auto'
    
    return 'color: %s' % color

def style(df):
    return df.style.applymap(color_null_red) 

# Join city and country data retrived from GeonamesCache to a 'data' dataframe 

In [284]:
cities = cities()

countries = countries()

cities shape = (24336, 8)


,geonameid,city,latitude,longitude,countrycode,population,timezone,admin1code
16890,3452237,Promissao,-2e+01,-5e+01,BR,"27,031",America/Sao_Paulo,27
2985,1816234,Bozhou,3e+01,1e+02,CN,"174,140",Asia/Shanghai,01
18771,3167509,Sant'Anastasia,4e+01,1e+01,IT,"20,733",Europe/Rome,04
22239,11838960,Valley Glen,3e+01,-1e+02,US,"60,000",America/Los_Angeles,CA
22662,472722,Volkhov,6e+01,3e+01,RU,"45,673",Europe/Moscow,42




countries shape = (252, 17)


,geonameid,country,iso,iso3,isonumeric,fips,continentcode,capital,areakm2,population,tld,currencycode,currencyname,phone,postalcoderegex,languages,neighbours
0,878675,Zimbabwe,ZW,ZWE,716,ZI,AF,Harare,"390,580","13,061,000",.zw,ZWL,Dollar,263,,"en-ZW,sn,nr,nd","ZA,MZ,BW,ZM"
1,895949,Zambia,ZM,ZMB,894,ZA,AF,Lusaka,"752,614","13,460,305",.zm,ZMW,Kwacha,260,^(\d{5})$,"en-ZM,bem,loz,lun,lue,ny,toi","ZW,TZ,MZ,CD,NA,MW,AO"
2,69543,Yemen,YE,YEM,887,YM,AS,Sanaa,"527,970","23,495,361",.ye,YER,Rial,967,,ar-YE,"SA,OM"
3,2461445,Western Sahara,EH,ESH,732,WI,AF,El-Aaiun,"266,000","273,008",.eh,MAD,Dirham,212,,"ar,mey","DZ,MR,MA"
4,4034749,Wallis and Futuna,WF,WLF,876,WF,OC,Mata Utu,274,"16,025",.wf,XPF,Franc,681,^(986\d{2})$,"wls,fud,fr-WF",


In [285]:
data = (cities[['city', 'population', 'countrycode', 'latitude', 'longitude']]
        .merge(countries[['country', 'population', 'capital', 'iso', 'fips','continentcode']], left_on='countrycode',
               right_on='iso',
               suffixes=('_city', '_country'),
               how='left',
               sort=True)
        .reset_index(drop=True))
data.info()
data.nunique()
# data.population_city = data.population_city.map('{:,.0f}'.format)
# data.population_country = data.population_country.map('{:,.0f}'.format)
style(data.sample(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24336 entries, 0 to 24335
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   city                24336 non-null  object
 1   population_city     24336 non-null  object
 2   countrycode         24336 non-null  object
 3   latitude            24336 non-null  object
 4   longitude           24336 non-null  object
 5   country             24336 non-null  object
 6   population_country  24336 non-null  object
 7   capital             24336 non-null  object
 8   iso                 24336 non-null  object
 9   fips                24336 non-null  object
 10  continentcode       24336 non-null  object
dtypes: object(11)
memory usage: 2.0+ MB


city                  23022
population_city       20035
countrycode             244
latitude              23702
longitude             23897
country                 244
population_country      244
capital                 243
iso                     244
fips                    243
continentcode             7
dtype: int64

,city,population_city,countrycode,latitude,longitude,country,population_country,capital,iso,fips,continentcode
12900,Izumo,"89,286",JP,35.4,132.8,Japan,"127,288,000",Tokyo,JP,JA,AS
10972,Nileshwar,"25,405",IN,12.3,75.1,India,"1,173,108,018",New Delhi,IN,IN,AS
1209,Potosi,"141,251",BO,-19.6,-65.8,Bolivia,"9,947,418",Sucre,BO,BL,SA
23585,West Mifflin,"20,075",US,40.4,-79.9,United States,"310,232,863",Washington,US,US,NA
16982,Nowy Sacz,"84,376",PL,49.6,20.7,Poland,"38,500,000",Warsaw,PL,PL,EU
9689,Balapur,"42,401",IN,20.7,76.8,India,"1,173,108,018",New Delhi,IN,IN,AS
13126,Niitsu-honcho,"65,910",JP,37.8,139.1,Japan,"127,288,000",Tokyo,JP,JA,AS
13457,Lamu,"24,525",KE,-2.3,40.9,Kenya,"40,046,566",Nairobi,KE,KE,AF
16688,Mailsi,"64,545",PK,29.8,72.2,Pakistan,"184,404,791",Islamabad,PK,PK,AS
3767,Tongchuanshi,"223,603",CN,35.1,109.1,China,"1,330,044,000",Beijing,CN,CH,AS


# Load city name dataset from the section 1

In [286]:
headline = pd.read_csv('1-cities_in_headline.csv')
print(headline.info())
style(headline.head())
headline.headline.value_counts()[headline.headline.value_counts()>1]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   headline  650 non-null    object
 1   city      616 non-null    object
 2   country   17 non-null     object
dtypes: object(3)
memory usage: 15.4+ KB
None


,headline,city,country
0,Zika Outbreak Hits Miami,Miami,nan
1,Could Zika Reach New York City?,New York City,nan
2,First Case of Zika in Miami Beach,Miami Beach,nan
3,"Mystery Virus Spreads in Recife, Brazil",Recife,Brazil
4,Dallas man comes down with case of Zika,Dallas,nan


Spanish Flu Spreading through Madrid    2
Spanish Flu Outbreak in Lisbon          2
Name: headline, dtype: int64

# join matched city name dataframe to 'data' dataframe 

In [287]:
joined = (headline.merge(data,
                         how='left',
                         left_on=[headline.city.str.lower()],
                         right_on=[data.city.str.lower()],
                         suffixes=('', '_ref'),
                         sort=False,                       
                         indicator=True,     
                         ))

print(f'{len(joined.headline.value_counts()[joined.headline.value_counts() > 1].index)}/650 headlines were redundant\n') 
print(f'\n{joined.info()}\n\tMissing value\n{joined.isnull().sum()}\n')
print(f'joined shape = {joined.shape}\n')
# style(joined[joined.country.notnull()])
style(joined[joined.city.isnull()])

174/650 headlines were redundant

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1003 entries, 0 to 1002
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   key_0               969 non-null    object  
 1   headline            1003 non-null   object  
 2   city                969 non-null    object  
 3   country             18 non-null     object  
 4   city_ref            964 non-null    object  
 5   population_city     964 non-null    object  
 6   countrycode         964 non-null    object  
 7   latitude            964 non-null    object  
 8   longitude           964 non-null    object  
 9   country_ref         964 non-null    object  
 10  population_country  964 non-null    object  
 11  capital             964 non-null    object  
 12  iso                 964 non-null    object  
 13  fips                964 non-null    object  
 14  continentcode       964 non-null    object  
 15  _mer

,key_0,headline,city,country,city_ref,population_city,countrycode,latitude,longitude,country_ref,population_country,capital,iso,fips,continentcode,_merge
108,nan,Zika case reported in Oton,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,left_only
129,nan,Maka City Experiences Influenza Outbreak,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,left_only
372,nan,Zika Virus Sparks 'International Concern',nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,left_only
386,nan,Greenwich Establishes Zika Task Force,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,left_only
393,nan,Will West Nile Virus vaccine help Parsons?,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,left_only
411,nan,Yulee takes a hit from Spreading Sickness,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,left_only
459,nan,Zika case reported in Los Fresnos,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,left_only
494,nan,More people in Boucau are infected with HIV every year,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,left_only
500,nan,How to Avoid Chlamydia in Santiago,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,left_only
522,nan,Bronchitis Outbreak in Manhasset,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,left_only


# filter dataframe to retain only relevent columns and drop all null rows 

In [288]:
joined.columns
# Create new working df and view info
df = joined[['headline', 'city', 'population_city', 'country', 'country_ref',
             'countrycode', 'population_country', 'latitude', 'longitude','continentcode']]
df.info()

# Number of Null rows in city columns
print(f'Number of Null rows in city columns {df.city.isnull().sum()}')

# filter df to view Null rows after joined
style(df[df.city.isnull()])

# Drop all 34 Null rows
df.drop(df[df.city.isnull()].index.tolist(), axis=0, inplace=True)
df.drop(df.city[df.countrycode.isnull()].index,axis=0,inplace=True)

Index(['key_0', 'headline', 'city', 'country', 'city_ref', 'population_city',
       'countrycode', 'latitude', 'longitude', 'country_ref',
       'population_country', 'capital', 'iso', 'fips', 'continentcode',
       '_merge'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1003 entries, 0 to 1002
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   headline            1003 non-null   object
 1   city                969 non-null    object
 2   population_city     964 non-null    object
 3   country             18 non-null     object
 4   country_ref         964 non-null    object
 5   countrycode         964 non-null    object
 6   population_country  964 non-null    object
 7   latitude            964 non-null    object
 8   longitude           964 non-null    object
 9   continentcode       964 non-null    object
dtypes: object(10)
memory usage: 86.2+ KB
Number of Null rows in city columns 34


,headline,city,population_city,country,country_ref,countrycode,population_country,latitude,longitude,continentcode
108,Zika case reported in Oton,nan,nan,nan,nan,nan,nan,nan,nan,nan
129,Maka City Experiences Influenza Outbreak,nan,nan,nan,nan,nan,nan,nan,nan,nan
372,Zika Virus Sparks 'International Concern',nan,nan,nan,nan,nan,nan,nan,nan,nan
386,Greenwich Establishes Zika Task Force,nan,nan,nan,nan,nan,nan,nan,nan,nan
393,Will West Nile Virus vaccine help Parsons?,nan,nan,nan,nan,nan,nan,nan,nan,nan
411,Yulee takes a hit from Spreading Sickness,nan,nan,nan,nan,nan,nan,nan,nan,nan
459,Zika case reported in Los Fresnos,nan,nan,nan,nan,nan,nan,nan,nan,nan
494,More people in Boucau are infected with HIV every year,nan,nan,nan,nan,nan,nan,nan,nan,nan
500,How to Avoid Chlamydia in Santiago,nan,nan,nan,nan,nan,nan,nan,nan,nan
522,Bronchitis Outbreak in Manhasset,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [289]:
from collections import Counter
# Number of redundancy headline
print(f'Number of redundancy headline: {len(df.headline.value_counts()[df.headline.value_counts() > 1])} ')
print(f'DataFrame Shape: {df.shape[0]} rows {df.shape[1]} columns ')

# Number of headlines group by number of redundancy rows
print('\nNumber of headlines group by number of redundancy rows')
Counter(df.headline.value_counts())

# Top 5 redundancy headlines
df.headline.value_counts().head()

Number of redundancy headline: 174 
DataFrame Shape: 964 rows 10 columns 

Number of headlines group by number of redundancy rows


Counter({8: 1, 7: 2, 6: 2, 5: 23, 4: 29, 3: 30, 2: 87, 1: 435})

Spike of Pneumonia Cases in Springfield                                    8
Lower Hospitalization in Richmond after Mumps Vaccine becomes Mandatory    7
Authorities are Worried about the Spread of Chickenpox in Richmond         7
Will Hepatitis B vaccine help La Paz?                                      6
Zika Virus Reaches San Francisco                                           6
Name: headline, dtype: int64

# Check the redundancy row of city columns 


In [290]:
# Number of redundancy row 
redun_headline = df.headline.value_counts()[df.headline.value_counts() > 1].index 
redun_index = {}
for x in df.headline:
    for y in df.headline.value_counts().index:
        if x == y:
            redun_index[x]=df[df.headline == x].index.tolist()
len(redun_index)
red_idx_val = list(redun_index.values())
length = [len(v) for v in red_idx_val]
length = sorted(length,reverse=True)
# Check count values
length == list(df.headline.value_counts().values)

609

True

# for all redundancy cities retain city that had largest population 

In [291]:
# add city population for each redundancy city in redun_index dictionary
for key, val in redun_index.items():
    n = 1
    while n < 8:
        if len(val) == n:
            new_val = val + ([np.nan] * (8-n))
        redun_index[key] = {'pop': df.loc[val, 'population_city']}
        n += 1
# for all redundancy cities retain city that had largest population
hl = list(redun_index.items())
for v in hl:
    hline, vdict = v
    if vdict['pop'].shape[0] > 1:
        for ind, pop in list(zip(vdict['pop'].index, vdict['pop'].values)):
            if pop != vdict['pop'].max():
                df.drop([ind], axis=0, inplace=True)

In [320]:
# df after remove all redundancy cities
df = df[['headline', 'city','latitude', 'longitude', 'countrycode','country_ref','continentcode']].reset_index(drop=True) 
df = df.rename({'country_ref':'country'},axis=1).dropna()
for i,s in enumerate(df.countrycode):
    if s == 'US':
        df.loc[i,'usa'] = True
    else:
        df.loc[i,'usa'] = False
        
df[['countrycode','usa']].sample(5)  

KeyError: "['country_ref'] not in index"

 # Check dup. and missing values again

In [296]:
df.info()
df.isnull().sum()
df.headline.value_counts()[:10]
style(df.loc[df.city.isnull(), :])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 611 entries, 0 to 610
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   headline       611 non-null    object
 1   city           611 non-null    object
 2   latitude       611 non-null    object
 3   longitude      611 non-null    object
 4   countrycode    611 non-null    object
 5   country        611 non-null    object
 6   continentcode  611 non-null    object
 7   usa            611 non-null    object
dtypes: object(8)
memory usage: 63.0+ KB


headline         0
city             0
latitude         0
longitude        0
countrycode      0
country          0
continentcode    0
usa              0
dtype: int64

Spanish Flu Spreading through Madrid                              2
Spanish Flu Outbreak in Lisbon                                    2
Zika Outbreak in South Miami                                      1
New Zika Case Confirmed in Belo Horizonte                         1
More Zika patients reported in Bella Vista                        1
Rumors about Mumps Spreading in New Bedford have been Refuted     1
Chikungunya Exposure in Toledo                                    1
Zika case reported in Phetchabun                                  1
Harare is infested with Pneumonia                                 1
Authorities are Worried about the Spread of Dengue in Kingston    1
Name: headline, dtype: int64

,headline,city,latitude,longitude,countrycode,country,continentcode,usa


## Vitualize countries 

In [297]:
df[df.countrycode != 'US'].country.plot(kind='hist',template='simple_white')

In [312]:
df.continentcode=df.continentcode.replace({'AF':'Africa',
                          'AS':'Asia',
                          'EU':'Europe',
                          'NA':'North America',
                          'OC':'Oceania',
                          'SA':'South America'})

df_n = df.groupby('continentcode').count()['country']

df_n.head()

df_n.values

continentcode
Africa            29
Asia              90
Europe            55
North America    364
Oceania           15
Name: country, dtype: int64

array([ 29,  90,  55, 364,  15,  58], dtype=int64)

In [313]:
px.pie(df_n,
     values=df_n.values,
     names=df_n.index,
     title='Percentage of countries related to diseases outbreak news headlines by continents')

In [317]:
result = df[['headline','city','latitude', 'longitude','countrycode']]
result

,headline,city,latitude,longitude,countrycode
0,Zika Outbreak Hits Miami,Miami,25.77,-80.19,US
1,Could Zika Reach New York City?,New York City,40.71,-74.01,US
2,First Case of Zika in Miami Beach,Miami Beach,25.79,-80.13,US
3,"Mystery Virus Spreads in Recife, Brazil",Recife,-8.054,-34.88,BR
4,Dallas man comes down with case of Zika,Dallas,44.92,-123.3,US
5,Trinidad confirms first Zika case,Trinidad,-14.83,-64.9,BO
6,Zika Concerns are Spreading in Houston,Houston,29.76,-95.36,US
7,Geneve Scientists Battle to Find Cure,Geneve,46.2,6.146,CH
8,The CDC in Atlanta is Growing Worried,Atlanta,33.75,-84.39,US
9,Zika Infested Monkeys in Sao Paulo,Sao Paulo,-23.55,-46.64,BR


In [319]:
result.to_csv('2-finding_geolocations.csv',index=False)